Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [13]:
train.shape

(4087, 3)

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
train.at[0, 'description']

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

In [29]:
# What is our majority baseline?
train['ratingCategory'].value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [14]:
target = 'ratingCategory'
features = 'description'
X_train = train[features]
y_train = train[target]
X_test = test[features]
#y_test = test[target]

### Define Pipeline Components

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [33]:
parameters = {
    'vect__max_df': (.75, 1.0),
    'vect__max_features': (1000, 2000, None),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators': (50, 100, 200)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-3)]: Done 180 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-3)]: Done 360 out of 360 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [34]:
grid_search.best_score_

0.7323223194165547

In [38]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 100,
 'vect__max_df': 0.75,
 'vect__max_features': 1000}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [35]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [36]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [39]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [27]:
subNumber = 0

In [40]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [41]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100,
                   algorithm='randomized',
                   n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lsi = Pipeline([('vect', vect), ('svd', svd)])

clf = RandomForestClassifier()
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'lsi__vect__max_features': (1000, 2000, None),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(X_train, y_train)

In [44]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7254754558540549
{'clf__max_depth': 20, 'lsi__svd__n_components': 100, 'lsi__vect__max_df': 0.75, 'lsi__vect__max_features': 2000}


### Make a Submission File

In [45]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [46]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [47]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [48]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [52]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [53]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [58]:
X = get_word_vectors(train['description'])

In [61]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

gbc = GradientBoostingClassifier()

param_dist = {
    'max_depth' : [3, 5, 7, 10],
    'min_samples_leaf': [2, 5, 8, 12]
}

grid_search = GridSearchCV(gbc, param_dist, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(X, train['ratingCategory'])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-3)]: Done  80 out of  80 | elapsed: 24.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [62]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7421040661014565
{'max_depth': 7, 'min_samples_leaf': 2}


### Make a Submission File

In [64]:
# Predictions on test sample
test_vectors = get_word_vectors(test['description'])
pred = grid_search.predict(test_vectors)
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [65]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [66]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

In [96]:
# A function for getting lemma tokens from review text
def get_lemmas(text):
    # Lemmatization
    lemmas = []    
    doc = nlp(text)
    for token in doc: 
        if (token.text.lower() not in nlp.Defaults.stop_words) and (token.is_punct == False) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_.lower())
    
    return lemmas

In [78]:
' '.join(get_lemmas(train.at[0, 'description']))

'\n whisky batch leftover barrel return warehouse canadian club recently pull vat 1970 acetone granny smith apple fresh cut white cedar showcase long age complex spicy reserve dram ripe strawberry can pear clove pepper faint flower slightly pull oak tannin distinct elegant remarkably vibrant ancient canadian club tired australia    a$ 133'

In [72]:
train.at[0, 'description']

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

In [ ]:
train['lemma_desc'] = train['description'].apply(lambda x : ' '.join(get_lemmas(x)))

In [109]:
def get_adjectives(text):
    # also performs lemmatization
    adjectives = []    
    doc = nlp(text)
    for token in doc: 
        if (token.pos_ == 'ADJ'):
            adjectives.append(token.lemma_.lower())
    return adjectives

In [110]:
train['adjectives'] = train['description'].apply(lambda x : ' '.join(get_adjectives(x)))
train.head()

,id,description,ratingCategory,lemma_desc,adjectives
0,1321,"\nSometimes, when whisky is batched, a few lef...",1,\n whisky batch leftover barrel return warehou...,few leftover several fresh white long complex ...
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,\n uncommon exclusive bottling 6 year old cask...,uncommon exclusive old light vegetal more odd ...
2,655,\nThis release is a port version of Amrut’s In...,1,\n release port version amrut intermediate she...,port unused few vibrant colorful complex wispy...
3,555,\nThis 41 year old single cask was aged in a s...,1,\n 41 year old single cask age sherry butt int...,old single red american balanced harmonious fr...
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1,\n herbal nose aroma dry tarragon parsley dill...,orange sweet full balanced orange candied yout...


### Let's try using those lemmas

In [82]:
# Define pipeline

# Latent semantic analysis
svd = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lsi = Pipeline([('vect', vect), ('svd', svd)])

#clf = RandomForestClassifier()
clf = GradientBoostingClassifier()
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [86]:
# Perfrorm grid search
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, .9),
    'lsi__vect__max_features': (2000, None),
    'clf__max_depth': (3,5,10,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(train['lemma_desc'], train['ratingCategory'])

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-3)]: Done 180 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-3)]: Done 430 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-3)]: Done 720 out of 720 | elapsed: 24.0min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [87]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7254736602693976
{'clf__max_depth': 20, 'clf__n_estimators': 100, 'lsi__svd__n_components': 100, 'lsi__vect__max_df': 0.9, 'lsi__vect__max_features': 2000}


### Pipeline using just the adjectives

In [113]:
# Define pipeline

# Latent semantic analysis
svd = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lsi = Pipeline([('vect', vect), ('svd', svd)])

#clf = RandomForestClassifier()
clf = GradientBoostingClassifier()
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [114]:
# Perfrorm grid search
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, .9),
    'clf__max_depth': (3,5,10,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(train['adjectives'], train['ratingCategory'])

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-3)]: Done 120 out of 120 | elapsed: 16.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [115]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7169048310205206
{'clf__max_depth': 10, 'lsi__svd__n_components': 100, 'lsi__vect__max_df': 0.9}


### Spacy lemma vectors

This is closest to my best result from before, the only difference is that I'm getting vectors of the lemmatized description instead of the full description

In [120]:
# Get the vectors of the lemmatized description
lemma_vectors = get_word_vectors(train['lemma_desc'])

In [121]:
# Apply to the dataset
gbc = GradientBoostingClassifier()

param_dist = {
    'max_depth' : [3, 5, 7, 10],
    'min_samples_leaf': [2, 5, 8, 12]
}

grid_search = GridSearchCV(gbc, param_dist, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(lemma_vectors, train['ratingCategory'])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-3)]: Done  80 out of  80 | elapsed: 23.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [122]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7232677845178704
{'max_depth': 5, 'min_samples_leaf': 2}


### Trying spacy adjective vectors

In [117]:
adj_vectors = get_word_vectors(train['adjectives'])

In [118]:
# Apply to the dataset
gbc = GradientBoostingClassifier()

param_dist = {
    'max_depth' : [3, 5, 7, 10],
    'min_samples_leaf': [2, 5, 8, 12]
}

grid_search = GridSearchCV(gbc, param_dist, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(adj_vectors, train['ratingCategory'])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-3)]: Done  80 out of  80 | elapsed: 23.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
         

In [119]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7252270666431245
{'max_depth': 10, 'min_samples_leaf': 12}


### Nearest Neighbors Classifier

In [127]:
from sklearn.neighbors import KNeighborsClassifier

knc = KNeighborsClassifier()

param_dist = {
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'n_neighbors' : [2, 5, 8]
}

grid_search = GridSearchCV(knc, param_dist, cv=5, n_jobs=-3, verbose=1)

# using the spacy vectors, already calculated
grid_search.fit(lemma_vectors, train['ratingCategory'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-3)]: Done  60 out of  60 | elapsed:   21.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-3,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [2, 5, 8]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [128]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.7044303058778465
{'algorithm': 'auto', 'n_neighbors': 8}


In [129]:
# Try again with the sklearn pipeline

# Latent semantic analysis
svd = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lsi = Pipeline([('vect', vect), ('svd', svd)])

knc = KNeighborsClassifier()
pipe = Pipeline([('lsi', lsi), ('knc', knc)])

In [130]:
# Grid search
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': [0.75, .9],
    'knc__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knc__n_neighbors' : [2, 5, 8]
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-3, verbose=1)
grid_search.fit(train['adjectives'], train['ratingCategory'])

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-3)]: Done  30 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-3)]: Done 180 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-3)]: Done 360 out of 360 | elapsed:  5.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [131]:
print("Best score:", grid_search.best_score_)
print(grid_search.best_params_)

Best score: 0.6696821515892422
{'knc__algorithm': 'brute', 'knc__n_neighbors': 8, 'lsi__svd__n_components': 10, 'lsi__vect__max_df': 0.9}


### Make a submission file

In [ ]:
# Perform same pre-processing
test['lemma_desc'] = test['description'].apply(lambda x : ' '.join(get_lemmas(x)))
test['adjectives'] = test['description'].apply(lambda x : ' '.join(get_adjectives(x)))

In [ ]:
# Predictions on test sample
test_vectors = get_word_vectors(test['adjectives'])
pred = grid_search.predict(test_vectors)

submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1